Open Play Shots - Expected Goal Model - Model Tuning and Building - GBM

In [52]:
import sys
sys.path.append("/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/")

In [53]:
import pandas as pd
import numpy as np
import seaborn as sns
import joblib
import warnings
warnings.filterwarnings('ignore')

from expected_score_model.config import open_shots_file_path
# from expected_score_model.domain.preprocessing.data_preprocessor import DataPreprocessor
# from expected_score_model.domain.contracts.mappings import Mappings
from expected_score_model.domain.contracts.modelling_data_contract import ModellingDataContract
from expected_score_model.domain.modelling.hyperparameter_tuning import XGBHyperparameterTuner
from expected_score_model.domain.modelling.supermodel import SuperXGBClassifier
from expected_score_model.domain.modelling.optuna_xgb_param_grid import OptunaXGBParamGrid

pd.options.display.max_rows = 100
pd.options.display.max_columns = 999

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Model Version

In [54]:
model_version = 4
model_name = 'expected_goal_open'
model_file_name = model_name + '_v' + str(model_version)

model_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/models"
prediction_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/predictions/"

preprocessor_file_name = 'preprocessor_v' + str(model_version)
preprocessor_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/preprocessors/"

In [55]:
RESPONSE = ModellingDataContract.RESPONSE_GOAL

In [ ]:
FEATURES = ModellingDataContract.open_goal_modelling_feature_list

Load Data

In [56]:
df_modelling = pd.read_csv(open_shots_file_path)
df_modelling.tail()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Goal,Behind,Miss,Score,Home_Score,Away_Score,Home_Current_Score,Away_Current_Score,Current_Margin,Event_Type0,Event_Type1,Event_Type2,Event_Type3,x0,x1,x2,x3,y0,y1,y2,y3,Quarter_Duration0,Quarter_Duration1,Quarter_Duration2,Quarter_Duration3,Chain_Duration,Time_Since_Last_Action,Distance_Since_Last_Action,Distance_to_Right_Goal_x,Distance_to_Middle_y,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Angle_to_Middle_Goal_degrees,Visible_Goal_Angle,Visible_Goal_Angle_degrees,Visible_Behind_Angle,Visible_Behind_Angle_degrees,Squared_Distance_to_Middle_Goal,Log_Distance_to_Middle_Goal,Set_Shot,random100,ModellingFilter
9211,211,kickIn,behind,1705.0,4,744,775.0,Sydney,Sydney,Tom Papley,Tom_Papley,Kick,56.0,-36.0,ineffective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,0,1,0,1,0,1,107.0,32.0,75.0,Kick,Gather,Kick,Handball Received,56.0,48.0,7.0,4.0,-36.0,-44.0,-47.0,-37.0,775.0,771.0,769.0,766.0,31.0,4.0,11.313708,24.0,36.0,43.266615,0.982794,56.309932,0.082503,4.727050,0.258900,14.833859,576.0,3.178054,False,16,True
9212,212,kickIn,goal,1740.0,4,787,900.0,Geelong,Geelong,Brandan Parfitt,Brandan_Parfitt,Kick,75.0,0.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,6,0,113.0,33.0,80.0,Kick,Handball Received,Handball,Loose Ball Get,75.0,75.0,76.0,75.0,0.0,-1.0,-5.0,-6.0,900.0,899.0,898.0,897.0,113.0,1.0,1.000000,5.0,0.0,5.000000,0.000000,0.000000,2.168022,124.218492,-1.429422,-81.899864,25.0,1.609438,False,43,True
9213,218,possGain,goal,1837.0,4,1136,1198.0,Geelong,Geelong,Sam De Koning,Sam_De_Koning,Kick,78.0,1.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,6,0,119.0,39.0,80.0,Kick,Gather,Kick,Handball Received,78.0,78.0,65.0,64.0,1.0,1.0,-12.0,-9.0,1198.0,1193.0,1190.0,1188.0,62.0,5.0,0.000000,2.0,1.0,2.236068,0.463648,26.565051,-2.442748,-139.959156,-0.440569,-25.242748,4.0,0.693147,False,5,True
9214,227,throwIn,goal,1901.0,4,1452,1458.0,Geelong,Geelong,Joel Selwood,Joel_Selwood,Kick,37.0,17.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,6,0,125.0,45.0,80.0,Kick,Gather,Knock On,Handball,37.0,37.0,43.0,34.0,17.0,19.0,20.0,25.0,1458.0,1457.0,1455.0,1453.0,6.0,1.0,2.000000,43.0,17.0,46.238512,0.376490,21.571307,0.129338,7.410516,0.403551,23.121747,1849.0,3.761200,False,88,False
9215,233,possGain,goal,1971.0,4,1744,1753.0,Sydney,Sydney,Tom Papley,Tom_Papley,Kick,48.0,-20.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,0,6,137.0,45.0,92.0,Kick,Bounce,Handball Received,Handball,48.0,38.0,35.0,37.0,-20.0,-31.0,-33.0,-37.0,1753.0,1752.0,1748.0,1747.0,9.0,1.0,14.866069,32.0,20.0,37.735925,0.558599,32.005383,0.144862,8.299977,0.461317,26.431498,1024.0,3.465736,False,23,True


In [57]:
df_modelling[['ballUp', 'centreBounce', 'kickIn', 'possGain', 'throwIn']] = pd.get_dummies(df_modelling['Initial_State'])

In [58]:
training_data = df_modelling[df_modelling[ModellingDataContract.TRAIN_TEST_SPLIT_COL]]
test_data = df_modelling[~df_modelling[ModellingDataContract.TRAIN_TEST_SPLIT_COL]]

In [59]:
X_train, y_train = training_data.drop(columns=[RESPONSE]), training_data[RESPONSE]
X_test, y_test = test_data.drop(columns=[RESPONSE]), test_data[RESPONSE]

In [60]:
X_test.head()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Behind,Miss,Score,Home_Score,Away_Score,Home_Current_Score,Away_Current_Score,Current_Margin,Event_Type0,Event_Type1,Event_Type2,Event_Type3,x0,x1,x2,x3,y0,y1,y2,y3,Quarter_Duration0,Quarter_Duration1,Quarter_Duration2,Quarter_Duration3,Chain_Duration,Time_Since_Last_Action,Distance_Since_Last_Action,Distance_to_Right_Goal_x,Distance_to_Middle_y,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Angle_to_Middle_Goal_degrees,Visible_Goal_Angle,Visible_Goal_Angle_degrees,Visible_Behind_Angle,Visible_Behind_Angle_degrees,Squared_Distance_to_Middle_Goal,Log_Distance_to_Middle_Goal,Set_Shot,random100,ModellingFilter,ballUp,centreBounce,kickIn,possGain,throwIn
2,10,possGain,goal,110.0,1,285,306.0,Brisbane Lions,Brisbane Lions,Joe Daniher,Joe_Daniher,Kick,50.0,24.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,0,0,6,6,0,12.0,0.0,12.0,Kick,Gather,Kick Inside 50 Result,Kick Into F50,50.0,50.0,50.0,-8.0,24.0,28.0,28.0,44.0,306.0,305.0,304.0,303.0,21.0,1.0,4.000000,28.0,24.0,36.878178,0.708626,40.601295,0.132764,7.606837,0.424028,24.295030,784.0,3.332205,False,83,False,0,0,0,1,0
7,50,possGain,behind,512.0,1,1462,1500.0,Brisbane Lions,Brisbane Lions,Oscar McInerney,Oscar_McInerney,Kick,65.0,-15.0,ineffective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,1,0,1,1,0,19.0,15.0,4.0,Kick,Hard Ball Get Crumb,Spoil,Kick Inside 50 Result,65.0,62.0,60.0,60.0,-15.0,-16.0,-17.0,-17.0,1500.0,1499.0,1497.0,1496.0,38.0,1.0,3.162278,13.0,15.0,19.849433,0.856706,49.085617,0.216802,12.421849,0.826928,47.379494,169.0,2.564949,False,90,False,0,0,0,1,0
8,79,possGain,goal,747.0,2,709,720.0,Sydney,Sydney,Sam Wicks,Sam_Wicks,Kick,41.0,18.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,0,0,6,0,6,20.0,40.0,-20.0,Kick,Handball Received,Handball,Handball Received,41.0,38.0,37.0,36.0,18.0,16.0,15.0,15.0,720.0,720.0,719.0,718.0,11.0,0.0,3.605551,37.0,18.0,41.146081,0.452778,25.942295,0.140721,8.062731,0.443767,25.425977,1369.0,3.610918,False,83,False,0,0,0,1,0
14,114,possGain,behind,1011.0,2,2025,2032.0,Brisbane Lions,Brisbane Lions,Joe Daniher,Joe_Daniher,Kick,54.0,35.0,ineffective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,1,0,1,1,0,50.0,52.0,-2.0,Kick,Loose Ball Get,Kick Inside 50 Result,Kick Into F50,54.0,49.0,41.0,-9.0,35.0,33.0,31.0,42.0,2032.0,2029.0,2027.0,2027.0,7.0,3.0,5.385165,24.0,35.0,42.438190,0.969723,55.561011,0.085774,4.914467,0.269657,15.450186,576.0,3.178054,False,87,False,0,0,0,1,0
15,123,centreBounce,goal,1086.0,3,128,143.0,Brisbane Lions,Brisbane Lions,Zac Bailey,Zac_Bailey,Kick,32.0,8.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,0,0,6,6,0,51.0,58.0,-7.0,Kick,Handball Received,Handball,Gather,32.0,22.0,26.0,26.0,8.0,18.0,20.0,20.0,143.0,140.0,139.0,139.0,15.0,3.0,14.142136,46.0,8.0,46.690470,0.172191,9.865807,0.135683,7.774075,0.423021,24.237313,2116.0,3.828641,False,92,False,0,1,0,0,0


Preprocess Data

In [61]:
# preprocessor = DataPreprocessor()
# preprocessor.fit(X_train)

# X_train_preproc = preprocessor.transform(X_train)
# X_test_preproc = preprocessor.transform(X_test)

In [62]:
X_train_preproc = X_train[FEATURES]
X_test_preproc = X_test[FEATURES]

In [63]:
X_train_preproc.shape, X_test_preproc.shape

((7208, 6), (2008, 6))

In [64]:
X_train_preproc.head()

,x0,Distance_Since_Last_Action,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,possGain
0,26.0,8.246211,56.080300,0.383818,0.106164,0
1,35.0,7.000000,47.010637,0.416065,0.125105,1
3,34.0,15.132746,46.486557,0.328553,0.130930,1
4,40.0,4.123106,39.560081,0.281772,0.156423,1
5,27.0,3.162278,59.682493,0.546167,0.091898,1


In [65]:
X_test_preproc.head()

,x0,Distance_Since_Last_Action,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,possGain
2,50.0,4.000000,36.878178,0.708626,0.132764,1
7,65.0,3.162278,19.849433,0.856706,0.216802,1
8,41.0,3.605551,41.146081,0.452778,0.140721,1
14,54.0,5.385165,42.438190,0.969723,0.085774,1
15,32.0,14.142136,46.690470,0.172191,0.135683,0


Optuna Hyperparameter Tuning Class - HyperParameterTuner & XGBHyperparameterTuner

In [66]:
xgb_tuner = XGBHyperparameterTuner(X_train_preproc, y_train, monotonicity_constraints=ModellingDataContract.monotone_constraints_open_goal)

In [67]:
xgb_tuner.training_data.head()

,x0,Distance_Since_Last_Action,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,possGain
0,26.0,8.246211,56.080300,0.383818,0.106164,0
1,35.0,7.000000,47.010637,0.416065,0.125105,1
3,34.0,15.132746,46.486557,0.328553,0.130930,1
4,40.0,4.123106,39.560081,0.281772,0.156423,1
5,27.0,3.162278,59.682493,0.546167,0.091898,1


In [68]:
xgb_tuner.tune_hyperparameters()

[I 2023-06-28 17:45:16,825] A new study created in memory with name: no-name-2eec840b-4411-4f10-ae2e-c0240aec584b
[I 2023-06-28 17:45:16,854] Trial 0 finished with value: 0.6713603644482483 and parameters: {'max_depth': 18, 'min_child_weight': 66, 'eta': 0.06020093827041626, 'gamma': 4.505104140318979, 'lambda': 0.03116498755318953, 'alpha': 6.932650071250086, 'subsample': 0.7080280631015787, 'colsample_bytree': 0.24458493514680796}. Best is trial 0 with value: 0.6713603644482483.
[I 2023-06-28 17:45:16,884] Trial 1 finished with value: 0.6300314722102844 and parameters: {'max_depth': 20, 'min_child_weight': 87, 'eta': 0.18183708815380153, 'gamma': 0.32793769053200017, 'lambda': 0.0014226041605817181, 'alpha': 0.18376688007839712, 'subsample': 0.6664918546137031, 'colsample_bytree': 0.8550483712654422}. Best is trial 1 with value: 0.6300314722102844.
[I 2023-06-28 17:45:16,924] Trial 2 finished with value: 0.6859699545327211 and parameters: {'max_depth': 14, 'min_child_weight': 53, 'et

Number of finished trials:  100
Best trial:
  Value: 0.6034906245451853
  Params: 
    max_depth: 11
    min_child_weight: 5
    eta: 0.49758909470280516
    gamma: 0.0003817001211350986
    lambda: 2.300474130992247
    alpha: 0.0005380136296108804
    subsample: 0.5293240301208615
    colsample_bytree: 0.5570102147755264


In [69]:
params = xgb_tuner.get_best_params()
params

{'max_depth': 11,
 'min_child_weight': 5,
 'eta': 0.49758909470280516,
 'gamma': 0.0003817001211350986,
 'lambda': 2.300474130992247,
 'alpha': 0.0005380136296108804,
 'subsample': 0.5293240301208615,
 'colsample_bytree': 0.5570102147755264}

Training Model - SuperXGBClassifier class for training and predictions

In [70]:
params['objective'] = OptunaXGBParamGrid.error
params['num_rounds'] = 1000
params['early_stopping_rounds'] = 50
params['verbosity'] = 1
params['monotone_constraints'] = ModellingDataContract.monotone_constraints_open_goal

In [71]:
super_xgb = SuperXGBClassifier(X_train = X_train_preproc, 
                               y_train = y_train, 
                               X_test = X_test_preproc, 
                               y_test = y_test, 
                               params = params)

In [72]:
super_xgb.fit()

[0]	validation_0-logloss:0.64783	validation_1-logloss:0.65261
[1]	validation_0-logloss:0.64292	validation_1-logloss:0.65012
[2]	validation_0-logloss:0.63231	validation_1-logloss:0.64259
[3]	validation_0-logloss:0.62258	validation_1-logloss:0.63604
[4]	validation_0-logloss:0.61890	validation_1-logloss:0.63421
[5]	validation_0-logloss:0.61451	validation_1-logloss:0.63060
[6]	validation_0-logloss:0.61344	validation_1-logloss:0.63033
[7]	validation_0-logloss:0.61294	validation_1-logloss:0.63042
[8]	validation_0-logloss:0.61279	validation_1-logloss:0.63137
[9]	validation_0-logloss:0.61260	validation_1-logloss:0.63176
[10]	validation_0-logloss:0.61079	validation_1-logloss:0.63035
[11]	validation_0-logloss:0.60995	validation_1-logloss:0.63083
[12]	validation_0-logloss:0.60953	validation_1-logloss:0.63177
[13]	validation_0-logloss:0.60908	validation_1-logloss:0.63270
[14]	validation_0-logloss:0.60919	validation_1-logloss:0.63398
[15]	validation_0-logloss:0.60880	validation_1-logloss:0.63382
[1

In [73]:
super_xgb.xgb_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.5570102147755264, early_stopping_rounds=50,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.0003817001211350986, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.49...
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=11, max_leaves=None,
              min_child_weight=5, missing=nan,
              monotone_constraints={'Angle_to_Middle_Goal': -1,
                                    'Distance_Since_Last_Action': 1,
                                    'Distance_to_Middle_Goal': -1,
                                    'Visible_Goal_Angle': 1, 'x0': -1},
              n_estimators=1000, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [74]:
super_xgb.xgb_model.get_booster().feature_names

['x0',
 'Distance_Since_Last_Action',
 'Distance_to_Middle_Goal',
 'Angle_to_Middle_Goal',
 'Visible_Goal_Angle',
 'possGain']

In [75]:
train_preds = super_xgb.predict(X_train_preproc)
test_preds = super_xgb.predict(X_test_preproc)

In [76]:
train_probas = super_xgb.predict_proba(X_train_preproc)[:, 1]
test_probas = super_xgb.predict_proba(X_test_preproc)[:, 1]

Check Average Predictions

In [77]:
train_probas.mean(), training_data[RESPONSE].mean(), train_probas.mean(), test_data[RESPONSE].mean()

(0.42516595, 0.4242508324084351, 0.42516595, 0.44123505976095617)

Export model

In [78]:
super_xgb.export_model(model_output_path + "/" + model_file_name + ".joblib")

Export data and predictions

In [79]:
train_info = training_data[['Match_ID', "Chain_Number", "Order", RESPONSE]].reset_index(drop = True)
train_export = pd.concat([train_info, X_train_preproc.reset_index(drop=True)], axis=1)
train_export['xgb_preds'] = train_preds
train_export['xgb_probas'] = train_probas
train_export.to_csv(prediction_output_path + 'train_predictions_' + model_file_name + '.csv', index = False)
train_export.head()

,Match_ID,Chain_Number,Order,Goal,x0,Distance_Since_Last_Action,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,possGain,xgb_preds,xgb_probas
0,202101_BrisbaneLions_Sydney,1,11.0,1,26.0,8.246211,56.080300,0.383818,0.106164,0,0,0.334652
1,202101_BrisbaneLions_Sydney,6,61.0,1,35.0,7.000000,47.010637,0.416065,0.125105,1,0,0.350948
2,202101_BrisbaneLions_Sydney,21,222.0,1,34.0,15.132746,46.486557,0.328553,0.130930,1,0,0.433789
3,202101_BrisbaneLions_Sydney,38,392.0,0,40.0,4.123106,39.560081,0.281772,0.156423,1,0,0.386675
4,202101_BrisbaneLions_Sydney,43,444.0,0,27.0,3.162278,59.682493,0.546167,0.091898,1,0,0.226262


In [80]:
test_info = test_data[['Match_ID', "Chain_Number", "Order", RESPONSE]].reset_index(drop = True)
test_export = pd.concat([test_info, X_test_preproc.reset_index(drop=True)], axis=1)
test_export['xgb_preds'] = test_preds
test_export['xgb_probas'] = test_probas
test_export.to_csv(prediction_output_path + 'test_predictions_' + model_file_name + '.csv', index = False)
test_export.head()

,Match_ID,Chain_Number,Order,Goal,x0,Distance_Since_Last_Action,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,possGain,xgb_preds,xgb_probas
0,202101_BrisbaneLions_Sydney,10,110.0,1,50.0,4.000000,36.878178,0.708626,0.132764,1,0,0.333238
1,202101_BrisbaneLions_Sydney,50,512.0,0,65.0,3.162278,19.849433,0.856706,0.216802,1,0,0.452057
2,202101_BrisbaneLions_Sydney,79,747.0,1,41.0,3.605551,41.146081,0.452778,0.140721,1,0,0.351711
3,202101_BrisbaneLions_Sydney,114,1011.0,0,54.0,5.385165,42.438190,0.969723,0.085774,1,0,0.265827
4,202101_BrisbaneLions_Sydney,123,1086.0,1,32.0,14.142136,46.690470,0.172191,0.135683,0,0,0.404775


Save preprocessor

In [ ]:
# joblib.dump(preprocessor, preprocessor_output_path + preprocessor_file_name + ".joblib")